In [41]:
import pymongo
import pickle
import pandas as pd
from ipywidgets import widgets

## Reading in Model from Pickle File

## Functions to query database

In [6]:
# Create client and load in Fraud database
client = pymongo.MongoClient()
db = client["Fraud"]
fraud_data = db["FraudData"]

In [7]:
def account_exists(account_name: str) -> bool:
    """
    Determine whether the given account exists in the database
    
    :param account_name: Name of the account in question
    :returns: Whether or not account exists in database
    """
    orig_account = fraud_data.find_one({"nameOrig": account_name})
    dest_account = fraud_data.find_one({"nameDest": account_name})
    
    return orig_account is not None or dest_account is not None

In [8]:
def get_account_balance(account_name: str) -> float:
    """
    Retrieve most recent final account balance for this account
    
    :param account_name: Name of the account in question
    :returns: Most recent account balance
    """
    all_trans = fraud_data.find({
        "$or": [{"nameOrig": account_name}, {"nameDest": account_name}]
    })
    all_trans = list(all_trans)
    
    # Return None if no prior transactions for this account
    if len(all_trans) == 0:
        return None
    # If account was an origin in most recent transaction, return balance after transaction
    elif all_trans[-1]["nameOrig"] == account_name:
        return all_trans[-1]["newbalanceOrig"]
    # Account was destination in most recent transaction, return balance after transaction
    else:
        return all_trans[-1]["newbalanceDest"]

## Function to predict whether transaction is fraudulent

In [51]:
def predict_fraud(trans: dict) -> int:
    """
    Predict whether the given transaction is fraudulent
    
    :param trans: Transaction to predict
    :returns: 1 if fraudulent, 0 if not
    """
    # Load model
    filename = 'DTFraudModel'
    loaded_model = pickle.load(open(filename, 'rb'))
    
    # These transactions cannot be fraudulent
    if trans["type"] in ["CASH_IN", "DEBIT", "PAYMENT"]:
        return 0

    features = [1 if trans["type"] == "TRANSFER" else 0, trans['amount'],trans['oldbalanceOrg'], \
                trans['newbalanceOrig'], trans['oldbalanceDest'], trans['newbalanceDest']]
    
    # Try to predict if transactional is fraudulent
    try:
        return loaded_model.predict([features])
    # Features contain a null value
    except ValueError:
        return 0
    
      

### Function to load transaction into database

In [44]:
def insert_transaction(trans_type: str, amount: float, orig_account_name: str, dest_account_name: str) -> dict:
    """
    Attempt to insert the given transaction into the database, predicting whether it is fraud or not
    
    :amount float: Transaction amount
    :param trans_type: Type of the transaction
    :param orig_account_name: Account name of transaction origin
    :param dest_account_name: Account name of transaction destination
    :returns: Dictionary of transaction detaila
    """
    # Ensure it is a valid transaction type
    if trans_type not in ["CASH_IN", "CASH_OUT", "DEBIT", "PAYMENT", "TRANSFER"]:
        raise ValueError(f"Invalid transaction type: {trans_type}")
    # Ensure it is a valid origin account
    if not account_exists(orig_account_name):
        raise ValueError(f"Account non-existent: {orig_account_name}")
    # Ensure it is a valid destination account
    if not account_exists(dest_account_name):
        raise ValueError(f"Account non-existent: {dest_account_name}")
    
    # Increase account balance if putting cash in
    if trans_type == "CASH_IN":
        amount *= -1
    
    
    old_bal_orig = get_account_balance(orig_account_name)
    old_bal_dest = get_account_balance(dest_account_name)
    
    new_transaction = {'step': 1,
                       'type': trans_type,
                       'amount': amount,
                       'nameOrig': orig_account_name,
                       'oldbalanceOrg': old_bal_orig,
                       'newbalanceOrig': old_bal_orig - amount if old_bal_orig else None,
                       'nameDest': dest_account_name,
                       'oldbalanceDest': old_bal_dest,
                       'newbalanceDest': old_bal_dest + amount if old_bal_dest else None,
                       'isFlaggedFraud': 1 if trans_type in ["CASH_OUT", "TRANSFER"] and amount >= 200000 else 0}
    
    # Predict whether transaction is fraudulent
    new_transaction["isFraud"] = predict_fraud(new_transaction)
    
    transaction_id = fraud_data.insert_one(new_transaction)
    
    return new_transaction

# Interactive Widgets

## Retrieving Account Balance

In [38]:
# Create the labels, text box, and button
account_label = widgets.Label("Account Name:")
account_text = widgets.Text()
balance_button = widgets.Button(description="Retrieve Balance")
balance_label = widgets.Label()

# Display widgets
display(account_label)
display(account_text)
display(balance_button)
display(balance_label)

def update_balance(button: widgets.widget_button.Button) -> None:
    """
    Update the account balance based off the given account
    
    :param button: Button that was clicked
    """
    # Retrieve account name of entered account
    acccount_name = account_text.value
    
    # Check if the account exists
    if account_exists(acccount_name):
        # Retrieve balance and set label value as balance
        account_balance = get_account_balance(account_text.value)
        balance_label.value = f"ACCOUNT BALANCE: ${account_balance}"
    else:
        balance_label.value = f"ACCOUNT NOT FOUND: {account_text.value}"

# Attach the update balance function to the button
balance_button.on_click(update_balance)

Label(value='Account Name:')

Text(value='')

Button(description='Retrieve Balance', style=ButtonStyle())

Label(value='')

## Inserting New Transaction

In [40]:
# Create the labels, text boxes, and button
orig_account_label = widgets.Label("Origin Account Name:")
orig_account_text = widgets.Text()
dest_account_label = widgets.Label("Destination Account Name:")
dest_account_text = widgets.Text()
trans_type_label = widgets.Label("Transaction Type:")
trans_type_dropdown = widgets.Dropdown(options=[("Transfer", "TRANSFER"),
                                               ("Payment", "PAYMENT"),
                                               ("Debit", "DEBIT"),
                                               ("Cash-Out", "CASH_OUT"),
                                               ("Cash-In", "CASH_IN")])
amount_label = widgets.Label("Amount:")
amount_input = widgets.BoundedFloatText(description="$", min=0, max=1000000)
trans_button = widgets.Button(description="Confirm Transaction")
trans_status = widgets.Label()

# Display widgets
display(orig_account_label)
display(orig_account_text)
display(dest_account_label)
display(dest_account_text)
display(trans_type_label)
display(trans_type_dropdown)
display(amount_label)
display(amount_input)
display(trans_button)
display(trans_status)

def update_trans(button: widgets.widget_button.Button) -> None:
    """
    Insert the transaction into database if both account exists, calculating fraud %
    
    :param button: Button that was clicked
    """
    if not account_exists(orig_account_text.value):
        trans_status.value = f"ACCOUNT NOT FOUND: {orig_account_text.value}"
    elif not account_exists(dest_account_text.value):
        trans_status.value = f"ACCOUNT NOT FOUND: {dest_account_text.value}"
    
    trans = insert_transaction(trans_type_dropdown.value, 
                               amount_input.value, 
                               orig_account_text.value, 
                               dest_account_text.value)
    fraud_per = trans["isFraud"]*100
    
    trans_status.value = f"TRANSACTION CONFIRMED ({fraud_per}% Fraud)"
        
    

# Attach the update balance function to the button
trans_button.on_click(update_trans)

Label(value='Origin Account Name:')

Text(value='')

Label(value='Destination Account Name:')

Text(value='')

Label(value='Transaction Type:')

Dropdown(options=(('Transfer', 'TRANSFER'), ('Payment', 'PAYMENT'), ('Debit', 'DEBIT'), ('Cash-Out', 'CASH_OUT…

Label(value='Amount:')

BoundedFloatText(value=0.0, description='$', max=1000000.0)

Button(description='Confirm Transaction', style=ButtonStyle())

Label(value='')